## Estudio Permutation Importance

¿Qué variables tienen mayor impacto en nuestras predicciones?

Permutation importance nos va a enseñar si nuestro modelo depende en alguna variable en concreto, y cuánto depende en ella.

In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('SerieTotal2016_ext_top100.csv')
data.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc,prcp,tmax,tmin,...,poi,baseball,tennis,basket,soccer,event,no2,co,pm2_5,o3
0,1018,47.610756,-122.341899,2016-01-02 08:00:00,Belltown,2,0.00,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
1,1018,47.610756,-122.341899,2016-01-02 09:00:00,Belltown,2,3.70,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
2,1018,47.610756,-122.341899,2016-01-02 10:00:00,Belltown,2,18.52,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
3,1018,47.610756,-122.341899,2016-01-02 11:00:00,Belltown,2,14.81,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
4,1018,47.610756,-122.341899,2016-01-02 12:00:00,Belltown,2,7.41,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0


In [44]:
data.describe()

,element_key,latitude,longitude,day_year,occupation_perc,prcp,tmax,tmin,air_temp,road_temp,poi,baseball,tennis,basket,soccer,event,no2,co,pm2_5,o3
count,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.000000,363744.0,363744.0,363744.000000,363744.000000,363744.000000,363744.000000
mean,41979.668481,47.629584,-122.337119,183.394563,28.010158,0.127522,17.147424,8.735752,14.707501,18.390786,0.499835,0.009996,0.009996,0.009996,0.0,0.0,56.645272,512.568059,5.630918,67.179701
std,27501.352417,0.024841,0.021419,105.015401,16.759651,0.255853,7.057592,4.538997,6.516853,9.494084,0.500001,0.099479,0.099479,0.099479,0.0,0.0,14.797166,194.778756,2.953534,16.314317
min,1018.000000,47.594155,-122.386288,2.000000,0.000000,0.000000,0.560000,-5.000000,-2.020000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,25.756000,229.000000,1.763158,10.000000
25%,16942.000000,47.613412,-122.346256,92.000000,15.620000,0.000000,12.220000,5.560000,9.640000,10.890000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,46.812000,400.750000,3.728571,56.000000
50%,35845.000000,47.620221,-122.333643,182.000000,27.270000,0.000000,17.220000,9.440000,14.400000,17.280000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,55.460000,458.000000,4.953333,67.600000
75%,69184.750000,47.659092,-122.319331,274.000000,39.130000,0.150000,22.220000,12.220000,19.330000,25.210000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,66.301333,572.500000,6.529412,77.000000
max,94602.000000,47.680744,-122.312603,366.000000,100.000000,1.750000,35.000000,20.560000,47.950000,47.950000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,107.473333,1431.250000,21.306250,116.000000


In [45]:
# Convertimos la variable objetivo (occupation_perc) en categórica, en concreto, la transformamos en 5 categorías.
data['occupation_perc'] = pd.qcut(x=data['occupation_perc'], q=5, labels=False)

In [46]:
y = (data['occupation_perc'])  
feature_names = [i for i in data.columns if data[i].dtype in [np.int64, np.float64]]
X = data[feature_names]
X = X.drop('occupation_perc', axis=1)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
my_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

In [47]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(my_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

Weight,Feature
0.0924 ± 0.0020,longitude
0.0794 ± 0.0015,road_temp
0.0558 ± 0.0019,element_key
0.0497 ± 0.0017,latitude
0.0445 ± 0.0018,air_temp
0.0372 ± 0.0021,poi
0.0294 ± 0.0026,tmax
0.0227 ± 0.0019,day_year
0.0141 ± 0.0017,tmin
0.0064 ± 0.0017,no2


Gracias a Permutation Importance vemos que nuestro modelo es mayoritariamente dependiente de las variables "longitude", "road_temp" y "element_key". Siendo las variables menos importantes para nuestra predicción "basket", "soccer" y "event".